In [13]:
from google.oauth2 import service_account
import argparse
import googlemaps
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import sys
import pandas as pd
import numpy as np
import time
import six
import csv

#Update the cell below with your client key 
gmaps=googlemaps.Client(key='your client key')

#Update the cell below with path to your file of your google api key
file_path='/Users/gary.zhou/Documents/wingstop/service_account.json'
credentials = service_account.Credentials.from_service_account_file(file_path)

scoped_credentials = credentials.with_scopes(['https://www.googleapis.com/auth/cloud-language'])
client= language.LanguageServiceClient(credentials=credentials)

# Get Reviews

In [16]:
def review(df):
        id_list=[]
        rate_list=[]
        text_list=[]
        df['state'] = df.formatted_address.apply(lambda x: x[-8:-6])
        
        for key in df['place_id'].tolist():
            result=gmaps.place(key)
            if 'reviews' in result['result']:
                for num in range(len(result['result']['reviews'])):
                        id_list.append(key)
                        rate_list.append(result['result']['reviews'][num]['rating'])
                        text_list.append(result['result']['reviews'][num]['text'])
        df_re=pd.DataFrame({'place_id':id_list,'individual_rating':rate_list,'text':text_list})
        df_final=pd.merge(df_re,df,on='place_id')
        return df_final

In [39]:
# The csv file should has 'place_id' column
# Update the cell below with path to your csv file
file_path='/Users/gary.zhou/Desktop/test.csv'
df=pd.read_csv(file_path)
df_reviews=review(df)
df_reviews


,place_id,individual_rating,text,formatted_address,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,...,opening_hours.open_now,permanently_closed,photos,plus_code.compound_code,plus_code.global_code,price_level,rating,reference,state,types
0,ChIJsROENXdtToYR3OCdGmg306s,5,When we arrived they told us they didn't have ...,"5601 S Hulen St, Fort Worth, TX 76132",32.667677,-97.399434,32.669051,-97.398446,32.666352,-97.401145,...,True,NaN,"[{'height': 1080, 'html_attributions': ['<a hr...","MJ92+36 Fort Worth, Texas",8644MJ92+36,2,3.6,CmRbAAAA_962epRqLREYN7pE-1jvx2q-McgXAcvcBTUNRY...,TX,"['bar', 'restaurant', 'food', 'point_of_intere..."
1,ChIJsROENXdtToYR3OCdGmg306s,2,We went to this location for an early dinner. ...,"5601 S Hulen St, Fort Worth, TX 76132",32.667677,-97.399434,32.669051,-97.398446,32.666352,-97.401145,...,True,NaN,"[{'height': 1080, 'html_attributions': ['<a hr...","MJ92+36 Fort Worth, Texas",8644MJ92+36,2,3.6,CmRbAAAA_962epRqLREYN7pE-1jvx2q-McgXAcvcBTUNRY...,TX,"['bar', 'restaurant', 'food', 'point_of_intere..."
2,ChIJsROENXdtToYR3OCdGmg306s,5,Went here for a late dinner. It was pretty bus...,"5601 S Hulen St, Fort Worth, TX 76132",32.667677,-97.399434,32.669051,-97.398446,32.666352,-97.401145,...,True,NaN,"[{'height': 1080, 'html_attributions': ['<a hr...","MJ92+36 Fort Worth, Texas",8644MJ92+36,2,3.6,CmRbAAAA_962epRqLREYN7pE-1jvx2q-McgXAcvcBTUNRY...,TX,"['bar', 'restaurant', 'food', 'point_of_intere..."
3,ChIJsROENXdtToYR3OCdGmg306s,5,Always great service. The bartenders & servers...,"5601 S Hulen St, Fort Worth, TX 76132",32.667677,-97.399434,32.669051,-97.398446,32.666352,-97.401145,...,True,NaN,"[{'height': 1080, 'html_attributions': ['<a hr...","MJ92+36 Fort Worth, Texas",8644MJ92+36,2,3.6,CmRbAAAA_962epRqLREYN7pE-1jvx2q-McgXAcvcBTUNRY...,TX,"['bar', 'restaurant', 'food', 'point_of_intere..."
4,ChIJsROENXdtToYR3OCdGmg306s,1,Today was the worst service I've ever had! I'v...,"5601 S Hulen St, Fort Worth, TX 76132",32.667677,-97.399434,32.669051,-97.398446,32.666352,-97.401145,...,True,NaN,"[{'height': 1080, 'html_attributions': ['<a hr...","MJ92+36 Fort Worth, Texas",8644MJ92+36,2,3.6,CmRbAAAA_962epRqLREYN7pE-1jvx2q-McgXAcvcBTUNRY...,TX,"['bar', 'restaurant', 'food', 'point_of_intere..."
5,ChIJ4WMUwnZiz4cRw420IlmbvHs,3,Food is good but quite pricey. A lot of the ch...,"2725 N Kansas Expy, Springfield, MO 65803",37.246732,-93.312720,37.248070,-93.309973,37.245370,-93.313636,...,True,NaN,"[{'height': 1080, 'html_attributions': ['<a hr...","6MWP+MW Springfield, Springfield Township, MO",86986MWP+MW,2,3.1,CmRbAAAAtrjY9rjSCFiTNOgzh1Pjt-UmYFVX-bdAohqi9A...,MO,"['bar', 'restaurant', 'food', 'point_of_intere..."
6,ChIJ4WMUwnZiz4cRw420IlmbvHs,5,Went here today the staff was very friendly an...,"2725 N Kansas Expy, Springfield, MO 65803",37.246732,-93.312720,37.248070,-93.309973,37.245370,-93.313636,...,True,NaN,"[{'height': 1080, 'html_attributions': ['<a hr...","6MWP+MW Springfield, Springfield Township, MO",86986MWP+MW,2,3.1,CmRbAAAAtrjY9rjSCFiTNOgzh1Pjt-UmYFVX-bdAohqi9A...,MO,"['bar', 'restaurant', 'food', 'point_of_intere..."
7,ChIJ4WMUwnZiz4cRw420IlmbvHs,5,Amazing! So many choices and not a bad price. ...,"2725 N Kansas Expy, Springfield, MO 65803",37.246732,-93.312720,37.248070,-93.309973,37.245370,-93.313636,...,True,NaN,"[{'height': 1080, 'html_attributions': ['<a hr...","6MWP+MW Springfield, Springfield Township, MO",86986MWP+MW,2,3.1,CmRbAAAAtrjY9rjSCFiTNOgzh1Pjt-UmYFVX-bdAohqi9A...,MO,"['bar', 'restaurant', 'food', 'point_of_intere..."
8,ChIJ4WMUwnZiz4cRw420IlmbvHs,1,TERRIBLE. I went with a party of like 10 peopl...,"2725 N Kansas Expy, Springfield, MO 65803",37.246732,-93.312720,37.248070,-93.309973,37.245370,-93.313636,...,True,NaN,"[{'height': 1080, 'html_attributions': ['<a hr...","6MWP+MW Springfield, Springfield Township, MO",86986MWP+MW,2,3.1,CmRbAAAAtrjY9rjSCFiTNOgzh1Pjt-UmYFVX-


# Get sentiments

In [35]:
def entity_sentiment_text(text):
    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    document = types.Document(
        content=text.encode('utf-8'),
        type=enums.Document.Type.PLAIN_TEXT)

    encoding = enums.EncodingType.UTF32
    if sys.maxunicode == 65535:
        encoding = enums.EncodingType.UTF16

    result = client.analyze_entity_sentiment(document, encoding)
    
    return result




def sentiment_analysis(df):
    sleep=0
    place_id=[]
    word=[]
    score=[]
    magnitude=[]
    in_rate=[]
    
    for row in df.itertuples():
        try:
            re= entity_sentiment_text(row.text)
            for entity in re.entities:
                if entity.sentiment.magnitude:
                    place_id.append(row.place_id)
                    word.append(entity.name)

                    if entity.sentiment.score:
                        score.append(entity.sentiment.score)
                    else:
                        score.append(np.NaN)
                    magnitude.append(entity.sentiment.magnitude)
                    in_rate.append(row.individual_rating)


            sleep+=1
            if sleep%10==0:
                time.sleep(3)
        except:
            pass
    df_re=pd.DataFrame({'place_id':place_id,'word':word,'score':score,'magnitude':magnitude,'individual_rating':in_rate})
    return(df_re)
#df_1=pd.read_csv('df_Pizza_Hut.csv')
#df_del=pd.read_csv('FINAL_Places_to_Remove.csv')
#del_list=df_del['place_id'].tolist()
#df=df_1[~df_1['place_id'].isin(del_list)]




In [37]:
# The code below may takes long time to run
df_reviews_dropna=df_reviews.dropna(subset=['text'])
df_sentiment=sentiment_analysis(df_reviews_dropna)
df_sentiment



,place_id,word,score,magnitude,individual_rating
0,ChIJsROENXdtToYR3OCdGmg306s,waiter,0.4,0.9,5
1,ChIJsROENXdtToYR3OCdGmg306s,service,0.9,0.9,5
2,ChIJsROENXdtToYR3OCdGmg306s,food,0.9,0.9,5
3,ChIJsROENXdtToYR3OCdGmg306s,one,NaN,0.7,2
4,ChIJsROENXdtToYR3OCdGmg306s,Megan,-0.3,1.8,2
5,ChIJsROENXdtToYR3OCdGmg306s,Food,0.9,0.9,2
6,ChIJsROENXdtToYR3OCdGmg306s,dinner,0.1,0.1,2
7,ChIJsROENXdtToYR3OCdGmg306s,way,-0.3,0.3,2
8,ChIJsROENXdtToYR3OCdGmg306s,experience,-0.5,0.5,2
9,ChIJsROENXdtToYR3OCdGmg306s,appetizer,0.7,0.7,2
